In [53]:
import os
import pandas as pd

In [2]:
cd ..

/home/evan/Documents/github/Feedlot


In [156]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

### load transformed chainlink price data from data/transformed/chainlink_prices folder

In [171]:
# get name of each file in chainlink_prices folder without the .csv
files = [f.split('.')[0] for f in os.listdir('data/transformed/chainlink_prices')]

# sort files alphabetically
files.sort()

### load cow_df dataset from data/transformed folder

In [174]:
# load cow swaps dataset
cow_df = pd.read_csv('data/transformed/cow_swaps.csv', low_memory=False)

In [175]:
# sort cow_df by timestamp
cow_df = cow_df.sort_values(by='timestamp')

In [176]:
# add new date column based off of timestamp conversion
cow_df['date'] = pd.to_datetime(cow_df['timestamp'], unit='s')

In [148]:
cow_df.head(5)

,timestamp,gasPrice,feeAmount,txHash,settlement,buyAmount,sellAmount,order,buyAmountEth,sellAmountEth,buyAmountUsd,sellAmountUsd,solvers,environment,name,active,sellTokenSymbol,sellTokenDecimals,buyTokenSymbol,buyTokenDecimals,date
177087,1628089484,55000000474,6776000000000000,0x08100e7ba81be84ee0bdce43db6640e2f992ec9991a740a689e97d20dea9dafa,0x08100e7ba81be84ee0bdce43db6640e2f992ec9991a740a689e97d20dea9dafa,1225092772597923267,100000000000000000,0x81dbb9631c0bec7d9762e170ff5feb2c9b366d5ca9502036d20d99c91cafb8bfb54872859733a3dfbb2c5401ac68cd9ca84b3cd1610ab36f,0.093887,0.100000,246.157990,264.102661,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,WETH,18,GNO,18,2021-08-04 15:04:44
44069,1628146744,44000000000,3663931816677019648,0x8772387c9e80b23080cba1ce607defc91f6680d5803f745dedbcc218e98562e8,0x8772387c9e80b23080cba1ce607defc91f6680d5803f745dedbcc218e98562e8,17868105041905652,17060511063645525663,0xee0db370db55b657a0a2e507249d6e17575196c82434dee380fc1922e94b37967b2e78d4dfaaba045a167a70da285e30e8fca196610b92dc,0.017868,0.022882,47.899028,61.470674,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,BNT,18,WETH,18,2021-08-05 06:59:04
148297,1628150635,53250000505,5000000000000000,0x5baf36801788b2d23d99b0797cf472922c0962644ef96109c6b204aba244ac68,0x5baf36801788b2d23d99b0797cf472922c0962644ef96109c6b204aba244ac68,26873570767257920454,15000000000000000,0xd77be88ba691a3e170a4d4b1643f4387b7d98b7c3d71706951b90e5db37e7954424a46612794dbb8000194937834250dc723ffa5610ba248,0.010036,0.015000,26.873571,40.164107,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,WETH,18,DAI,18,2021-08-05 08:03:55
107549,1628151011,57333334166,1699,0x9cb8bb8c8227e893c3a5be72ffb093dc02c4ef154cdc24bfa9a90bc4f67b4734,0x9cb8bb8c8227e893c3a5be72ffb093dc02c4ef154cdc24bfa9a90bc4f67b4734,4262565514161118,2849,0xb69bc1be0b676c615f729d73a90cd111adab5e31d1c8dd64fecd5be146c7cb86424a46612794dbb8000194937834250dc723ffa5610ba3c1,0.004263,0.010530,11.413476,28.427503,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,GUSD,2,NaN,18,2021-08-05 08:10:11
79953,1628151263,46333333517,15247857004985886720,0x0143df0301e2b14420109dd63073450e71a1277f585d9680cedbe9a632860097,0x0143df0301e2b14420109dd63073450e71a1277f585d9680cedbe9a632860097,18683766405690006,65472865720793357605,0xb8d7f2b053ce3ce02468153cb2fd40c3c68cf8405e7f8aa6adb8eec6de5870678bc0fe84022542e1611ff252090028df741ad670610ba4ad,0.018684,0.024445,50.077756,65.472866,0x6c2999b6b1fad608ecea71b926d68ee6c62beef8,barn,Legacy,False,DAI,18,NaN,18,2021-08-05 08:14:23


### do transformations


In [177]:
# get name of each file in chainlink_prices folder without the .csv
files = [f.split('.')[0] for f in os.listdir('data/transformed/chainlink_prices')]

# sort files alphabetically
files.sort()

In [178]:
dfs_data_list = []

In [180]:
counter = 0

for name in files:
    print(f'Round {counter}')
    # load price data from chainlink_prices folder
    chain_df = pd.read_csv(f'data//transformed/chainlink_prices/{name}.csv')

    # convert chain_df_sorted to datetime
    chain_df['date'] = pd.to_datetime(chain_df['timestamp'], unit='s')
    
    # sort by timestamp
    chain_df_sorted = chain_df.sort_values(by='timestamp')


    # check which rows of buyTokenSymbol and sellTokenSymbol are COMP
    cow_buy = cow_df[cow_df['buyTokenSymbol'] == name] # mask filter
    cow_sell = cow_df[cow_df['sellTokenSymbol'] == name] # mask filter

    # sort cow_buy by timestamp and plot
    cow_buy_sorted = cow_buy.sort_values(by='date')
    cow_sell_sorted = cow_sell.sort_values(by='date')

    # left inner join chaink_df with cow_buy on date. Interpolate if date does not match up
    cow_chain_buy = pd.merge_asof(cow_buy_sorted, chain_df_sorted, on='date', direction='nearest', tolerance=pd.Timedelta('1d'), allow_exact_matches=False)
    # left inner join chaink_df with cow_sell on date. Interpolate if date does not match up
    cow_chain_sell = pd.merge_asof(cow_sell_sorted, chain_df_sorted, on='date', direction='nearest', tolerance=pd.Timedelta('1d'), allow_exact_matches=False)

    dfs_data_list.append(cow_chain_buy)
    dfs_data_list.append(cow_chain_sell)

    counter += 1

Round 0
Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
Round 51
Round 52
Round 53
Round 54
Round 55
Round 56
Round 57
Round 58
Round 59
Round 60
Round 61
Round 62
Round 63
Round 64
Round 65
Round 66
Round 67
Round 68
Round 69
Round 70
Round 71
Round 72
Round 73
Round 74
Round 75
Round 76
Round 77
Round 78
Round 79
Round 80
Round 81
Round 82
Round 83
Round 84
Round 85
Round 86
Round 87
Round 88
Round 89
Round 90
Round 91
Round 92
Round 93
Round 94
Round 95
Round 96
Round 97
Round 98
Round 99
Round 100
Round 101
Round 102
Round 103
Round 104
Round 105


In [183]:
# combine all dataframes in dfs_data_list into one dataframe
cow_chain_df = pd.concat(dfs_data_list)

In [185]:
cow_chain_df.head(5)

,timestamp_x,gasPrice,feeAmount,txHash,settlement,buyAmount,sellAmount,order,buyAmountEth,sellAmountEth,buyAmountUsd,sellAmountUsd,solvers,environment,name,active,sellTokenSymbol,sellTokenDecimals,buyTokenSymbol,buyTokenDecimals,date,id,assetPair_id,timestamp_y,price
0,1628686039,68931631460,2360000000000000,0xcd9c41a627319943bb238344b67d36acbb2f16cb440da54ac8d9c079f15702ba,0xcd9c41a627319943bb238344b67d36acbb2f16cb440da54ac8d9c079f15702ba,52380057311070130920,50000000000000000,0xa933a70023b2ebdfdc30a57579dbb37c21ece321d918e6833dbc36e8599aef8f581ea42c3ec995d9adb6f4239db2218b95fc85706113cb45,0.047599,0.050000,153.333211,161.487795,0xa6ddbd0de6b310819b49f680f65871bee85f517e,prod,Legacy,False,WETH,18,1INCH,18,2021-08-11 12:47:19,1INCH/USD/0x6113c8ac,1INCH/USD,1.628687e+09,2.955511
1,1628731364,54487387731,1977848014600000,0x4f70ddfc5b6ebd6095d1a615efcdf71c050236e5487c0157f88b717194e7134b,0x4f70ddfc5b6ebd6095d1a615efcdf71c050236e5487c0157f88b717194e7134b,81288008233613861906,78000000000000000,0x3e66589cefba0b75671e712d7849200a7c8def3e011eb45079f0b65988d456b8a314921b83db11d8139eb977880809b96bb09d1e61147c7a,0.076141,0.078000,243.277727,249.311263,0xa6ddbd0de6b310819b49f680f65871bee85f517e,prod,Legacy,False,WETH,18,1INCH,18,2021-08-12 01:22:44,1INCH/USD/0x61147610,1INCH/USD,1.628731e+09,3.005516
2,1628740359,48372879565,1877333333333333,0x6aa734b3570fabb0c3b14ef39376de36087a6dbf4a0c0c0aea880b17e2e65b7e,0x6aa734b3570fabb0c3b14ef39376de36087a6dbf4a0c0c0aea880b17e2e65b7e,100000000000000000,1958985990547882,0xf2ff56471575ca1d06eb7bc0130e318f879e9805eaf139209d00a237dedb2e5f204827ed867d1c9f8e632e695bfe0d57030ec4c66114a1d7,0.000094,0.001959,0.303182,6.324492,0xa6ddbd0de6b310819b49f680f65871bee85f517e,prod,Legacy,False,WETH,18,1INCH,18,2021-08-12 03:52:39,1INCH/USD/0x6114a0bc,1INCH/USD,1.628742e+09,3.004150
3,1628742997,43752398553,1734030229173333,0x1236d49b9b9b134ba00f42b6d924d21aa62fc244c9fbd1c8908fd03fab5ccd43,0x1236d49b9b9b134ba00f42b6d924d21aa62fc244c9fbd1c8908fd03fab5ccd43,9914254163616960274,10000000000000000,0x70c49bba41abd9055cbd464cfda10ca04a06021c6730b465b93468cd289c0ed37fe187c20a8b31b116651f52eeb8d848c456227d6114a9f9,0.009325,0.010000,29.782462,31.884226,0xa6ddbd0de6b310819b49f680f65871bee85f517e,prod,Legacy,False,WETH,18,1INCH,18,2021-08-12 04:36:37,1INCH/USD/0x6114a33d,1INCH/USD,1.628742e+09,2.973845
4,1628749848,39666666770,1625385267280000,0x2db0dc8ebc8d56a96389650e1b25103b1d92f9a17bdad6fd7ae71c07d4f90f21,0x2db0dc8ebc8d56a96389650e1b25103b1d92f9a17bdad6fd7ae71c07d4f90f21,2771933476979543757,4000000000000000,0x7393866196d9fbf3f9ff53be341926fec14aa789ac0a90255904418469f660e0a495893ceb72ae222d1e34d2dc1872b83c47d5fa6114c49f,0.002521,0.004000,7.868888,12.486523,0xa6ddbd0de6b310819b49f680f65871bee85f517e,prod,Legacy,False,WETH,18,1INCH,18,2021-08-12 06:30:48,1INCH/USD/0x6114c10d,1INCH/USD,1.628750e+09,2.811691


In [186]:
# save cow_chain_df to csv
cow_chain_df.to_csv('data/transformed/cow_chain_swaps.csv', index=False)